### Practical no 7 : Client & server, By Parth Gawad, Roll no : 62

#### Server file :

In [ ]:
import socket
import threading

PORT = 5050
SERVER = socket.gethostbyname(socket.gethostname())  # Get the local IP address of the host machine
ADDR = (SERVER, PORT)
FORMAT = 'utf-8'
HEADER = 64
DISCONNECT_MESSAGE = '!DISCONNECT'

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind(ADDR)

def client_handling(connection, address):
    print(f'[NEW CONNECTION] {address} connected')
    is_connected = True
    while is_connected:
        try:
            # Receive the length of the incoming message
            message_length = connection.recv(HEADER).decode(FORMAT)
            if message_length:
                message_length = int(message_length)  # Convert message length to an integer
                message = connection.recv(message_length).decode(FORMAT)
                
                if message == DISCONNECT_MESSAGE:
                    is_connected = False
                print(f'[{address}] {message}')
                
                # Send acknowledgment to the client
                connection.send('Message Received'.encode(FORMAT))
        except Exception as e:
            print(f'[ERROR] Error handling message from {address}: {e}')
            break
    
    connection.close()
    print(f'[DISCONNECTED] {address} disconnected')

def start():
    server.listen()
    print(f'[LISTENING] Server is listening on {SERVER}')
    
    while True:
        try:
            connection, address = server.accept()
            thread = threading.Thread(target=client_handling, args=(connection, address))
            thread.start()
            print(f'[ACTIVE CONNECTIONS] {threading.active_count() - 1}')
        except Exception as e:
            print(f'[ERROR] Error accepting a new connection: {e}')

print('[SERVER STARTING] Server has been started')
start()


[SERVER STARTING] Server has been started
[LISTENING] Server is listening on 192.168.174.1


#### Client File

In [2]:
import socket

PORT = 5050
SERVER = 'localhost'  # Update with the actual server address or IP
ADDR = (SERVER, PORT)
FORMAT = 'utf-8'
HEADER = 64
DISCONNECT_MESSAGE = '!DISCONNECT'

client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect(ADDR)

def send(message):
    message = message.encode(FORMAT)
    messageLength = len(message)
    sendLength = str(messageLength).encode(FORMAT)
    sendLength += b' ' * (HEADER - len(sendLength))
    client.send(sendLength)
    client.send(message)
    
    # Receive the response from the server
    response = client.recv(2048).decode(FORMAT)
    print(f"Server response: {response}")

if __name__ == '__main__':
    # Sending the "Hello World" message
    send("Hello World")
    
    # Sending the disconnect message
    send(DISCONNECT_MESSAGE)

    # Close the client socket after the communication
    client.close()


ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it